In [1]:
import pandas as pd
import yfinance as yf       # https://pypi.org/project/yfinance/
import pytse_client as tse
import numpy as np

/opt/homebrew/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
def _get_tse_prices(tickers):
    prices_dict = tse.download(symbols=tickers, adjust=True)
    prices_dict_reform = {(outerKey, innerKey):
                                values for outerKey, innerDict 
                                in prices_dict.items() for innerKey, values 
                                in innerDict.iteritems()}
    df_raw = pd.DataFrame(prices_dict_reform)

    dates_df = df_raw.loc[:, df_raw.columns.get_level_values(1)=='date']
    min_date, max_date = np.min(np.min(dates_df)), np.max(np.max(dates_df))

    df = pd.DataFrame()
    df['Date'] = pd.date_range(min_date, max_date)

    for symbol in df_raw.columns.levels[0]:
        df_tmp = df_raw[symbol]
        df_tmp = df_tmp[['date', 'adjClose']]
        df_tmp.columns = ['Date', symbol]
        df = df.merge(df_tmp, on='Date', how='left')

    df = df.set_index('Date')

    return df

In [4]:
def get_prices(tickers, is_tse=False):
    
    if is_tse:
        return _get_tse_prices(tickers)
    else:    
        df_raw = yf.download(tickers, group_by = 'ticker', period = '100y', interval='1d')
        df = df_raw.loc[:, df_raw.columns.get_level_values(1)=='Adj Close']
        df.columns = df.columns.levels[0]
        return df

,حفاری,حکشتی,خودرو,وبصادر,وسینا,وپارس,کرماشا
Date,,,,,,,
2001-03-25,NaN,NaN,54.0,NaN,NaN,NaN,NaN
2001-03-26,NaN,NaN,54.0,NaN,NaN,NaN,NaN
2001-03-27,NaN,NaN,54.0,NaN,NaN,NaN,NaN
2001-03-28,NaN,NaN,55.0,NaN,NaN,NaN,NaN
2001-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2022-12-31,4199.0,26570.0,2821.0,1639.0,2299.0,1725.0,63970.0
2023-01-01,4056.0,26520.0,2934.0,1600.0,2267.0,1662.0,63080.0
2023-01-02,4141.0,27830.0,3074.0,1636.0,2298.0,1696.0,62850.0


In [16]:
tickers = ['وسينا',
 'حكشتي',
 'وپارس',
 'وبصادر',
 'حفاري',
 'كرماشا',
 'خودرو',]

In [ ]:
[go]

In [14]:
tickers = ['وسینا',
 'حکشتی',
 'وپارس',
 'وبصادر',
 'حفاری',
 'کرماشا',
 'خودرو',]

In [26]:
def groom(s):
    s = s.replace('ي', 'ی')
    # s = s.replace('ک', 'ک')
    s = s.replace('ی', 'ی')
    s = s.replace('ك', 'ک')
    return s

In [27]:
tickers = [groom(x) for x in tickers]

In [28]:
get_prices(tickers, is_tse=True)

,حفاری,حکشتی,خودرو,وبصادر,وسینا,وپارس,کرماشا
Date,,,,,,,
2001-03-25,NaN,NaN,54.0,NaN,NaN,NaN,NaN
2001-03-26,NaN,NaN,54.0,NaN,NaN,NaN,NaN
2001-03-27,NaN,NaN,54.0,NaN,NaN,NaN,NaN
2001-03-28,NaN,NaN,55.0,NaN,NaN,NaN,NaN
2001-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2022-12-31,4199.0,26570.0,2821.0,1639.0,2299.0,1725.0,63970.0
2023-01-01,4056.0,26520.0,2934.0,1600.0,2267.0,1662.0,63080.0
2023-01-02,4141.0,27830.0,3074.0,1636.0,2298.0,1696.0,62850.0
